<a href="https://colab.research.google.com/github/stanleykywu/Facial_Hapiness_Sadness_Detection/blob/master/FaceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [0]:
from google.colab import drive
drive.mount("/content/drive")

In [0]:
import os
import cv2

train_path = "/content/drive/My Drive/Colab Notebooks/train/"
test_path = "/content/drive/My Drive/Colab Notebooks/test/"

train_file_names = os.listdir(train_path)
test_file_names = os.listdir(test_path)

def load_image(file_path):
    return cv2.imread(file_path)

def assign_label(file_name):
  if "happy" in file_name:
    return 1
  else:
    return 0

train_images = [load_image(train_path + file) for file in train_file_names]
train_labels = [assign_label(file) for file in train_file_names]

print(train_file_names)

In [0]:
def gray_scale(img, side=500):
    #min_side = min(img.shape[0], img.shape[1])
    #img = img[:min_side, :min_side]
    img = cv2.resize(img, (side,side*2))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img / 255.0

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

preview_index = 11
plt.subplot(1,2,1)
plt.imshow(train_images[preview_index])
plt.subplot(1,2,2)
plt.imshow(gray_scale(train_images[preview_index]), cmap="gray")

In [0]:
for i in range(len(train_images)):
  train_images[i] = gray_scale(train_images[i])

In [0]:
import numpy as np

train_as_array = np.expand_dims(train_images, axis=-1)
train_labels_as_array = np.array(train_labels)
print(train_as_array.shape, train_labels_as_array.shape)

(20, 1000, 500, 1) (20,)


In [0]:
import tensorflow as tf
print("Tensorflow:", tf.__version__)

layers = [
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding="same", activation=tf.nn.relu, input_shape=train_as_array.shape[1:]),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=2, activation=tf.nn.softmax)
]

model = tf.keras.Sequential(layers)
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])
model.fit(train_as_array, train_labels_as_array, epochs=70, batch_size=50)
model.save_weights("model.tf")

In [0]:
from google.colab import files
uploads = files.upload()

In [0]:
uploads.keys()

In [0]:
test_images = [gray_scale(load_image(file)) for file in uploads.keys()]
eval_model = tf.keras.Sequential(layers)
eval_model.load_weights("model.tf")
eval_predictions = eval_model.predict(np.expand_dims(test_images, axis=-1))

cols = 4
rows = np.ceil(len(test_images)/cols)
fig = plt.gcf()
fig.set_size_inches(cols * 4, rows * 4)
for i in range(len(test_images)):
    plt.subplot(rows, cols, i+1)
    plt.imshow(test_images[i], cmap="gray")
    plt.title("Happy" if np.argmax(eval_predictions[i])==1 else "Sad")
    plt.axis('off')